In [1]:
from collections import namedtuple
import altair as alt
import math
import pandas as pd
import streamlit as st
from transformers import BertModel, BertTokenizer
from transformers import pipeline
import requests
from bs4 import BeautifulSoup
import re
from transformers import BartTokenizer, BartForConditionalGeneration
import torch

In [45]:
def summarize_text(url,model, tokenizer):
    # facebook/bart-large-xsum
    
    # https://arxiv.org/abs/1910.13461
    url = url
    response = requests.get(url)
    # Create a BeautifulSoup object to parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')
    text = soup.text
    #transformed_text = re.sub(r'[^ ]', r'\\', text)
    max_length = 1024

    text_len = len(text)

    n = round(text_len/max_length)
    # Create text chunk
    chunk=[]
    for i in range(n):
        start = (i*max_length)
        end = (i+1)*max_length
        if end > text_len:
            end = text_len
            
        chunk.append(text[start:end])

    #print(truncated_input)
    # Transform input tokens 

    result_text = []
    for text in chunk:
        inputs = tokenizer.encode(text, return_tensors="pt")

        # Generate summary
        summary_ids = model.generate(inputs, num_beams=4, max_length=1024, early_stopping=True)

        # Convert summary IDs to text
        summary_text = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)

        result_text.append(summary_text)

    concatenated_text = ''.join(result_text)

    return concatenated_text

In [46]:
model_name = "facebook/bart-large-xsum" 
# Download pytorch model
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

In [47]:
url ='https://www.anywise.com.au/services'

In [48]:
summary = summarize_text(url,model,tokenizer)

In [ ]:
summary

In [5]:
response = requests.get(url)
# Create a BeautifulSoup object to parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')
text = soup.text

In [6]:
transformed_text =  re.sub(r'\s', ' ', text.strip())

In [38]:
text_len = len(transformed_text)

n = round(text_len/1024)

chunk=[]
for i in range(n):
    start = (i*1024)
    end = (i+1)*1024
    chunk.append(transformed_text[start:end])
  

16